In [1]:
from correctingagent.world.world import *
from correctingagent.agents.teacher import *
from correctingagent.world.rules import Rule
import os
import correctingagent.pddl.ff as ff

from pgmpy.factors.discrete import TabularCPD

from correctingagent.models.pgmmodels import PGMModel, PGMPYInference
from correctingagent.models.prob_model import KDEColourModel
from correctingagent.world import rules
from correctingagent.world.rules import RedOnBlueRule, CorrectionType


In [11]:
pgm_model = PGMModel()

red_cm = KDEColourModel('red')
blue_cm = KDEColourModel('blue')

time = 0
red_on_blue_rules = rules.Rule.generate_red_on_blue_options('red', 'blue')

violations = pgm_model.extend_model(red_on_blue_rules, red_cm, blue_cm, ['b1', 'b2'], time, correction_type=CorrectionType.TOWER)



inference = PGMPYInference(pgm_model.model)
new_model = inference.infer({}, {'F(b1)':[1,1,1], 'F(b2)':[0,0,0], f'corr_{time}':1})
q = inference.query(violations)
#
# q = pgm_model.query(violations, [1, 1])

assert(q[violations[0]] == 0.5)
assert(q[violations[1]] == 0.5)


fix bw? False
bw 0.5
fix bw? False
bw 0.5


In [7]:
f = q[violations[0]]

In [13]:
f.values[0]

0.5

In [3]:
inference.belief_propagation.map_query(variables=['V_0(all x.(red(x) -> exists y. (blue(y) & on(x,y))))'], evidence=inference.evidence)

{'V_0(all x.(red(x) -> exists y. (blue(y) & on(x,y))))': 0}

In [3]:
model = inference.new_model

AttributeError: 'PGMPYInference' object has no attribute 'new_model'

In [5]:
new_model

In [12]:
a = pgm_model.model.factors[0]
print(a.scope())
b = a.reduce([('F(b1)', [1,1,1])], inplace=False)
print(b.scope())
print(type(b))
c = b.reduce([('red(b1)', 1)], inplace=False)
print(c.scope())
print(type(c))
g = a.reduce([('red(b1)', 1)], inplace=False)

d = g.reduce([('F(b1)', [1,1,1])], inplace=False)

['F(b1)', 'red(b1)']
['red(b1)']
<class 'pgmpy.factors.continuous.ContinuousFactor.ContinuousFactor'>
[]
<class 'pgmpy.factors.continuous.ContinuousFactor.ContinuousFactor'>


In [18]:
d.distribution._pdf()

TypeError: <lambda>() missing 2 required positional arguments: 'rgb' and 'c'

In [27]:
pgm_model.observe({'F(b1)':[1,1,1], 'F(b2)':[0,0,0], f'corr_{time}':1})
pgm_model.infer()

/home/mappelgren/Desktop/pgmpy/pgmpy/factors/discrete/DiscreteFactor.py:621: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  phi.values = phi.values[slice_]
/home/mappelgren/Desktop/pgmpy/pgmpy/factors/discrete/DiscreteFactor.py:633: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  phi1.values = phi1.values[slice_]


In [13]:
pgm_model.query(variables=violations)

{'V_0(all x.(red(x) -> exists y. (blue(y) & on(x,y))))': 0.5,
 'V_0(all y.(blue(y) -> exists x. (red(x) & on(x,y))))': 0.5}

In [14]:
inference.evidence

{'corr_0': 1}

In [5]:
from pgmpy.factors.discrete import TabularCPD
from pgmpy.models import BayesianModel
from pgmpy.inference import BeliefPropagation
bayesian_model = BayesianModel([('A', 'J'), ('R', 'J'), ('J', 'Q'),
                                 ('J', 'L'), ('G', 'L')])
cpd_a = TabularCPD('A', 2, [[0.2], [0.8]])
cpd_r = TabularCPD('R', 2, [[0.4], [0.6]])
cpd_j = TabularCPD('J', 2,
                    [[0.9, 0.6, 0.7, 0.1],
                     [0.1, 0.4, 0.3, 0.9]],
                    ['R', 'A'], [2, 2])
cpd_q = TabularCPD('Q', 2,
                    [[0.9, 0.2],
                     [0.1, 0.8]],
                    ['J'], [2])
cpd_l = TabularCPD('L', 2,
                    [[0.9, 0.45, 0.8, 0.1],
                     [0.1, 0.55, 0.2, 0.9]],
                    ['G', 'J'], [2, 2])
cpd_g = TabularCPD('G', 2, [[0.6], [0.4]])
bayesian_model.add_cpds(cpd_a, cpd_r, cpd_j, cpd_q, cpd_l, cpd_g)
belief_propagation = BeliefPropagation(bayesian_model)
bp = belief_propagation.map_query(variables=['J', 'Q'],
                              evidence={'A': 0, 'R': 0, 'G': 0, 'L': 1})


0

In [3]:
a = {1:2}

In [4]:
a.update({2:3})

In [5]:
a

{1: 2, 2: 3}